<a href="https://colab.research.google.com/github/svezden/Machine-Learning/blob/master/Model_arrows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional neural network for the classification of arrows

In this notebook we will create a model that will learn to identify different arqueological artifacts assigning to them a particular period where they where constructed.

## Importing Tensorflow

In [4]:
import tensorflow as tf
print(tf.__version__)

1.14.0


Defining local directory where the photos will be stored. The zip files training and validation should contain the training and validation sets

In [0]:
import os
import zipfile

local_zip = '/tmp/arrows' # Here we define the local directory where the photos will be stored
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/arrows')
local_zip = '/tmp/validation-arrows.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-arrows')
zip_ref.close()

The contents of the .zip are extracted to the base directory `/tmp/arrows`. Each subdirectory should contain the different labels of the set.

### Keep an eye:
We do not explicitly label the images. We use ImageGenerator instead. This is coded to read images from subdirectories, and automatically label them from the name of that subdirectory. So, for example, you will have a 'training' directory containing a 'period1' directory and a 'period2' one. ImageGenerator will label the images appropriately using the name of the subdirectory they are in. 

Let's define each of these directories:

In [0]:
# Directory with our training horse pictures
train_period1_dir = os.path.join('/tmp/arrow/period1')

# Directory with our training human pictures
train_period2_dir = os.path.join('/tmp/arrow/period2')

# And so on, you have to fill in the rest....

# Directory with our training horse pictures
validation_period1_dir = os.path.join('/tmp/validation-arrows/validation-period1')

# Directory with our training human pictures
validation_period2_dir = os.path.join('/tmp/validation-arrows/validation-period2')


# And so on .....

Class to predefine a desired accuracy. This will help to stop training once the threshold accuracy has been reached.

In [0]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.99):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

###Model building.

We will use a convolutional neural network as an arquitecture, with three convolutional layers. After we get a handle on the performance of the model we can change the number of layers and the parameters in each layer. 
We have to build the model according to the number of periods that we want to predict. This parameter is N_periods 

In [0]:
N_periods = # This number enters as a parameter in the model

In [0]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # N_periods output neurons corresponding to the different periods that we want to capture
    tf.keras.layers.Dense(N_periods, activation='softmax')
])

model.summary()

Compiling the model we define the loss function tha measures how well the model approximates the data, and the type off optimizer to perform some of the many variants of gradient descent

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

### Data Preprocessing

Let's set up data generators that will read pictures in our source folders, convert them to `float32` tensors, and feed them (with their labels) to our network. We'll have one generator for the training images and one for the validation images. Our generators will yield batches of images of size 300x300 and their labels (binary).

As is common, we feed the model with the values of the pixel in the image scaled to be between 0 and 1.

In Keras this can be done via the `keras.preprocessing.image.ImageDataGenerator` class using the `rescale` parameter. This `ImageDataGenerator` class allows you to instantiate generators of augmented image batches (and their labels) via `.flow(data, labels)` or `.flow_from_directory(directory)`. These generators can then be used with the Keras model methods that accept data generators as inputs: `fit_generator`, `evaluate_generator`, and `predict_generator`.

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/arrows/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=128,
        # To handle multiple labels
        class_mode='categorical')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-arrows/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        # To handle multiple labels
        class_mode='categorical')

### Training the model
All the parameters here can be adjusted for better performance

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

Running the Model

This code allows us to choose 1 or more files from the file system, it will then upload them, and run them through the model, giving an indication of the period of the object.

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
 